In [4]:
import torch
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import DefaultDataCollator
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModelForCausalLM, AutoModelForSeq2SeqLM
from transformers import TrainingArguments, Trainer

In [5]:
if torch.cuda.is_available():
    device = "cuda"
    print("cuda")
else:
    device = "cpu"
    print("cpu")

cuda


In [12]:
tokenizer = AutoTokenizer.from_pretrained("riotu-lab/ArabianGPT-01B")
model = AutoModelForCausalLM.from_pretrained("riotu-lab/ArabianGPT-01B").to(device)
modelQA = AutoModelForQuestionAnswering.from_pretrained("riotu-lab/ArabianGPT-01B").to(device)

tokenizer_config.json:   0%|          | 0.00/106 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/546M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/94.0 [00:00<?, ?B/s]

Some weights of GPT2ForQuestionAnswering were not initialized from the model checkpoint at riotu-lab/ArabianGPT-01B and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [228]:
# Example prompt
prompt = "ان تكون أو لا تكون"

# Tokenize input
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

output = model.generate(
    input_ids,
    max_length=100,
    temperature=0.7,
    do_sample=True
)

# Decode and print generated text
generated_text = tokenizer.decode(output[0])
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:64000 for open-end generation.


ان تكون أو لا تكون قد سبق لها الحصول على إجازة دراسية من احدى الجامعات أو إحدى الجامعات السعودية.. الخ.. إلخ.. الخ.. وأن تكون قد مضى على حصولها على دبلوم المعاهد الفنية ثلاث سنوات على الأقل.. وأن يتم تحديد مدة الدراسة في المعاهد الفنية التجارية التابعة لإدارة التربية والتعليم بالمحافظة وفقا لنظام التعليم الفني والتدريب المهني.. وأن لا تقل عن ثلاث سنوات.. وأن لا تقل في مدة الدراسة عن ست سنوات.. وأن لا تقل في مدة الدراسة عن ست سنوات.. وأن لا


# Data Exploring

In [6]:
original_dataset = load_dataset("arcd")

Generating train split:   0%|          | 0/693 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/702 [00:00<?, ? examples/s]

In [7]:
print(original_dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 693
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 702
    })
})


In [8]:
# Extract the train and validation datasets
train_dataset = original_dataset['train']
val_dataset = original_dataset['validation']

# Select the first 200 rows for validation
new_val_dataset = val_dataset.select(range(100))

# Select the remaining rows for training
remaining_val_dataset = val_dataset.select(range(100, len(val_dataset)))


new_train_dataset = concatenate_datasets([train_dataset, remaining_val_dataset])

# Create a new DatasetDict with the updated splits
new_dataset = DatasetDict({
    'train': new_train_dataset,
    'validation': new_val_dataset
})

# Print the information of the new DatasetDict
print(new_dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 1295
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 100
    })
})


In [9]:
new_dataset = new_dataset.remove_columns(["id", "title"])
print(new_dataset)

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answers'],
        num_rows: 1295
    })
    validation: Dataset({
        features: ['context', 'question', 'answers'],
        num_rows: 100
    })
})


In [10]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="longest",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0] if answer["answer_start"] else None
        end_char = start_char + len(answer["text"][0]) if start_char is not None else None
        sequence_ids = inputs.sequence_ids(i) if inputs.sequence_ids(i) else []

        if not start_char or not end_char or not sequence_ids:
            # Handle cases where start_char, end_char, or sequence_ids are empty
            start_positions.append(0)
            end_positions.append(0)
            continue  # Move to the next iteration

        idx = 0
        while idx < len(sequence_ids) and sequence_ids[idx] != 1:
            idx += 1
        context_start = idx

        while idx < len(sequence_ids) and sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        if context_start >= len(offset) or context_end >= len(offset):
            # Handle cases where context indices exceed offset_mapping length
            start_positions.append(0)
            end_positions.append(0)
            continue  # Move to the next iteration

        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [13]:
processed_dataset = new_dataset.map(preprocess_function, batched=True,remove_columns=new_dataset["train"].column_names)

Map:   0%|          | 0/1295 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [14]:
print(processed_dataset)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 1295
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 100
    })
})


# Model Building

In [25]:
train_params = TrainingArguments(
    
    # Learning algorithms parameters
    optim = "paged_adamw_32bit",
    learning_rate = 3e-4,
    weight_decay = 0.01,
    lr_scheduler_type = 'cosine',
    
    # Memory Optimization parameters
    gradient_accumulation_steps = 4,
    gradient_checkpointing = True,
    
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    auto_find_batch_size='power_of_two',
    
    max_grad_norm=2,
    group_by_length=True,
    num_train_epochs=3,
    
    output_dir = '/model_outputs',
    logging_dir="./logs",
    save_steps = 50,
    logging_steps = 10,
    
    evaluation_strategy = "steps",  # Set the evaluation strategy to "steps"
    save_strategy = "steps",
    push_to_hub=True
)

In [26]:
data_collator = DefaultDataCollator(return_tensors="pt")

In [242]:
trainer = Trainer(
    model=modelQA,
    args=train_params,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [243]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
10,1.842200,3.557961
20,1.622500,3.497063
30,1.289700,3.732351
40,1.385500,3.406400
50,0.647000,4.031374
60,1.130500,3.619244
70,0.914300,3.592458
80,0.878400,3.568894
90,0.397200,3.691824
100,0.345500,3.762138


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=117, training_loss=0.9433518699091724, metrics={'train_runtime': 259.0401, 'train_samples_per_second': 14.419, 'train_steps_per_second': 0.452, 'total_flos': 731957539806720.0, 'train_loss': 0.9433518699091724, 'epoch': 3.0})

In [248]:
trainer.save_model("/kaggle/working/final_model")

In [275]:
final_model = AutoModelForQuestionAnswering.from_pretrained("/kaggle/working/final_model/")

In [320]:
def answer_question(context, question):

  # Preprocess the context and question
  inputs = tokenizer(
    question,
    context,
    max_length=384,
    truncation="only_second",
    return_tensors="pt",
    )

  # Generate the answer using the model
  outputs = final_model(**inputs)
  start_logits = outputs.start_logits
  end_logits = outputs.end_logits

  
  answer_start = torch.argmax(start_logits, dim=-1).item()
  answer_end = torch.argmax(end_logits, dim=-1).item()

  answer = tokenizer.decode(inputs.input_ids[0][answer_start:answer_end + 1])
  return answer

# Example usage
def answer_question(context, question):

  # Preprocess the context and question
  inputs = tokenizer(
    question,
    context,
    max_length=384,
    truncation="only_second",
    return_tensors="pt",
    )

  # Generate the answer using the model
  outputs = final_model(**inputs)
  start_logits = outputs.start_logits
  end_logits = outputs.end_logits

  
  answer_start = torch.argmax(start_logits, dim=-1).item()
  answer_end = torch.argmax(end_logits, dim=-1).item()

  answer = tokenizer.decode(inputs.input_ids[0][answer_start:answer_end + 1])
  return answer

# Example usage
context = "تتأثر الألياف العصبية الطويلة بدرجة أكبر من الألياف العصبية القصيرة، وذلك لأن سرعة التوصيل في العصب تنقص في تناسب مع طول العصب. في هذه المتلازمة، يحدث انخفاض في الإحساس وفقدان ردود الفعل في أصابع كل قدم، وتمتد بعد ذلك إلى أعلى. وعادة ما توصف باحساس الخدر وفقدان الإحساس وعسر اللمس (انخفاض أو فقدان الإحساس في جزء من الجسم) وألم ليلي فيما يشبه القفاز والجورب. ويمكن أن يكون الألم في هيئة حرقان أو وخز أو ألم غير محدد. ويكون الاحساس بوخز الدبابيس والإبر أمراً شائعاً. ويتأثر الاحساس بوضع أعضاء الجسم لبعضها proprioception مبكرا. ولا يمكن لهؤلاء المرضى الشعور عندما يدوسون على جسم غريب كالشظية، أو عندما يتكون لهم جلد صلب من الأحذية الضيقة. وبناء على ذلك، فإنهم معرضون لخطر حدوث القرحة والتهابات القدمين والساقين، والتي يمكن أن تؤدي إلى البتر وقد يحدث لهؤلاء المرضى كسورا متعددة في الركبة أو الكاحل أو القدم وقد تؤدي إلى حدوث انحلال في المفاصل. ويؤدي فقدان وظيفة الحركة إلى تقوس القدم لأعلى dorsiflexion، وتقلص أصابع القدم وفقدان وظيفة العضلات بين الأصابع، مما يسمى بالقدم المطرقة. ولا تقتصر هذه التقلصات على القدم فقط، بل أيضا تصيب اليد حيث فقدان العضلات يجعل اليد تبدو هزيلة كالهيكل العظمي ويزداد فقدان الوظيفة الحركية"
question = "ما هي العوامل التي تؤثر على سرعة الموصلات العصبية؟"

predicted_answer = answer_question(context, question)
print(f"Predicted Answer: {predicted_answer}")

Predicted Answer: سرعة التوصيل في العصب تنقص في تناسب مع طول العصب.


# ArabianGPT 0.3B

In [28]:
tokenizer = AutoTokenizer.from_pretrained("riotu-lab/ArabianGPT-03B")
model = AutoModelForCausalLM.from_pretrained("riotu-lab/ArabianGPT-03B").to(device)
modelQA = AutoModelForQuestionAnswering.from_pretrained("riotu-lab/ArabianGPT-03B").to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of GPT2ForQuestionAnswering were not initialized from the model checkpoint at riotu-lab/ArabianGPT-03B and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [333]:
# Example prompt
prompt = "ان تكون أو لا تكون"

# Tokenize input
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

output = model.generate(
    input_ids,
    max_length=100,
    temperature=0.8,
    do_sample=True
)

# Decode and print generated text
generated_text = tokenizer.decode(output[0])
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:64000 for open-end generation.


ان تكون أو لا تكون.. انا وانت.. لا ادري.. ؟!!.. لكن هذا هو الواقع.. ؟!!.. هل نحن على الطريق الصحيح لنرتقي ونتقدم ونصل الى ما نصبوا اليه ؟!!.. اما ان نكون مجرد اوراق في مهب الريح..!!.. نعم.. اننا على طريق الضياع.. الضياع.. الضياع الذي لا نعرف ماذا نريد.. ما نريده فقط هو ان نكون مجرد أوراق.. اوراق..


In [30]:
trainer = Trainer(
    model=modelQA,
    args=train_params,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
10,6.451500,4.585489
20,4.891200,4.160758
30,4.352400,4.050859
40,4.153700,4.048408
50,3.671600,4.021072
60,3.428400,4.135720
70,3.521500,4.252016
80,3.433600,4.027042


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


In [300]:
trainer.save_model("/kaggle/working/final_model_03")

model.safetensors:   0%|          | 0.00/1.48G [00:00<?, ?B/s]

In [317]:
final_model_03 = AutoModelForQuestionAnswering.from_pretrained("/kaggle/working/final_model_0.3")

In [318]:
def answer_question(context, question):

  # Preprocess the context and question
  inputs = tokenizer(
    question,
    context,
    max_length=384,
    truncation="only_second",
    return_tensors="pt",
    )

  # Generate the answer using the model
  outputs = final_model_03(**inputs)
  start_logits = outputs.start_logits
  end_logits = outputs.end_logits

  
  answer_start = torch.argmax(start_logits, dim=-1).item()
  answer_end = torch.argmax(end_logits, dim=-1).item()

  answer = tokenizer.decode(inputs.input_ids[0][answer_start:answer_end + 1])
  return answer

# Example usage
context = "تتأثر الألياف العصبية الطويلة بدرجة أكبر من الألياف العصبية القصيرة، وذلك لأن سرعة التوصيل في العصب تنقص في تناسب مع طول العصب. في هذه المتلازمة، يحدث انخفاض في الإحساس وفقدان ردود الفعل في أصابع كل قدم، وتمتد بعد ذلك إلى أعلى. وعادة ما توصف باحساس الخدر وفقدان الإحساس وعسر اللمس (انخفاض أو فقدان الإحساس في جزء من الجسم) وألم ليلي فيما يشبه القفاز والجورب. ويمكن أن يكون الألم في هيئة حرقان أو وخز أو ألم غير محدد. ويكون الاحساس بوخز الدبابيس والإبر أمراً شائعاً. ويتأثر الاحساس بوضع أعضاء الجسم لبعضها proprioception مبكرا. ولا يمكن لهؤلاء المرضى الشعور عندما يدوسون على جسم غريب كالشظية، أو عندما يتكون لهم جلد صلب من الأحذية الضيقة. وبناء على ذلك، فإنهم معرضون لخطر حدوث القرحة والتهابات القدمين والساقين، والتي يمكن أن تؤدي إلى البتر وقد يحدث لهؤلاء المرضى كسورا متعددة في الركبة أو الكاحل أو القدم وقد تؤدي إلى حدوث انحلال في المفاصل. ويؤدي فقدان وظيفة الحركة إلى تقوس القدم لأعلى dorsiflexion، وتقلص أصابع القدم وفقدان وظيفة العضلات بين الأصابع، مما يسمى بالقدم المطرقة. ولا تقتصر هذه التقلصات على القدم فقط، بل أيضا تصيب اليد حيث فقدان العضلات يجعل اليد تبدو هزيلة كالهيكل العظمي ويزداد فقدان الوظيفة الحركية"
question = "ما هي العوامل التي تؤثر على سرعة الموصلات العصبية؟"

predicted_answer = answer_question(context, question)
print(f"Predicted Answer: {predicted_answer}")

Predicted Answer: سرعة التوصيل في العصب تنقص في تناسب مع طول العصب.


# Generative QA Experiement

In [288]:
final_model_03_gen = AutoModelForCausalLM.from_pretrained("/kaggle/working/final_model_0.3")
final_model_gen = AutoModelForCausalLM.from_pretrained("/kaggle/working/final_model")

In [290]:
# Example question without context
question = "من هو جمال خاشقجي"

# Prepare the prompt (adjust as needed)
prompt = f"Question: {question} \n Answer: "

# Tokenize the prompt
input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"]

# Generate the answer using the model (adjust parameters like max_length)
with torch.no_grad():
    outputs = final_model_gen.generate(
        input_ids=input_ids,
        max_length=128,
        early_stopping=True,
        temperature=0.8,
        do_sample=True
    )

# Decode the generated tokens
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Generated Answer: {answer}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:64000 for open-end generation.


Generated Answer: Question: من هو جمال خاشقجي 
 Answer: ديث ).. في هذا الحوار.. عن هذا اللقاء.. ما هي اللقاءات الإعلامية في هذا اللقاء.. ما هي اللقاءات الإعلامية مع جمال خاشقجي ؟.. سؤال عن اللقاء.. هل كان للقاءك مع جمال خاشقجي ان يكون في هذا اللقاء الأخير ؟.. لا استطيع أن اجيب عن هذا السؤال.. ولكن هذا السؤال.. هل كان اللقاء الأخير مع جمال خاشقجي ؟.. اللقاء الأخير معك كان في هذا اللقاء.. هل كان للقاء الأخير أم لكومدارس اللقاء الأخير في اللقاء الأخير ؟.. هل كان اللقاء الأول للقاءاتك


In [19]:
from huggingface_hub import notebook_login

notebook_login()